In [ ]:
import requests
import random
import time
import json
import pycouchdb

base_url = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Covid19_RKI_Sums/FeatureServer/0/query?where=IdBundesland%3D11+AND+Meldedatum+%3D+TIMESTAMP+%27{}+00%3A00%3A00%27&objectIds=&time=&resultType=standard&outFields=&returnIdsOnly=true&returnUniqueIdsOnly=false&returnCountOnly=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&sqlFormat=standard&f=pjson&token="
feature_url = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/ArcGIS/rest/services/Covid19_RKI_Sums/FeatureServer/0/{}?f=pjson"

db_url = ""
server = pycouchdb.Server(db_url)
db = server.database("berlin_covid_sums")

def add_leading_zero(string):
    if len(string) < 2:
        return "0" + string
    return string

def find_documents_by_ids(ids):
    documents = []
    for i in ids:
        url = feature_url.format(i)
        feature = json.loads(requests.get(url).text)
        documents.append(feature)
    return documents

def sum_cases_and_deaths(documents):
    cases, deaths = 0, 0
    for document in documents:
        cases += document["feature"]["attributes"]["AnzahlFall"]
        deaths += document["feature"]["attributes"]["AnzahlTodesfall"]
        time.sleep(random.uniform(0,1))
    return cases, deaths

def post_to_db(month, day, cases, deaths):
    date = "{}.{}.2020".format(day, month)    
    return db.save({"date": date, "cases": cases, "deaths": deaths})

doc_ids = []

# First Covid-19 cases from API in March
for month in range(3, 13):
    for day in range(1, calendar.monthrange(2012,1)[1]+1):        
        m, d = add_leading_zero(str(month)), add_leading_zero(str(day))        
        timestamp = "2020-{}-{}".format(m, d)
        
        try:
            url = base_url.format(timestamp)
            data = json.loads(requests.get(url).text)
            
            documents = find_documents_by_ids(data["objectIds"])
            cases, deaths = sum_cases_and_deaths(documents)
            
            doc = post_to_db(m, d, cases, deaths)
            doc_ids.append(doc["_id"])
            print(timestamp, cases, deaths)
        except Exception:
            pass